# 🖐️ HandSpeak.ai - Production Training Pipeline

**Objective**: Train a Transformer-based ASL Character Recognition model.
**Input**: 30-frame sequences of 21 MediaPipe hand landmarks (x,y,z).

## Features
- **Robust Preprocessing**: Wrist centralization, Scale Normalization, Rotation Alignment.
- **Data Augmentation**: Simulates user variance (Rotation, Scale, Noise) and converts static frames to temporal sequences.
- **Class Balancing**: Handles dataset imbalance using Weighted Loss.
- **Visualization**: Verifies data quality and plots confusion matrices.
- **Production Export**: Generates ONNX model for mobile deployment.

## 1. Setup & Environment

In [ ]:
import os
import json
import random
import math
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob
from collections import Counter

# ----------------------------
# CONFIGURATION
# ----------------------------
CONFIG = {
    "FRAMES": 30,
    "LANDMARKS": 21,
    "DIMS": 3,
    "NUM_CLASSES": 26,
    "BATCH_SIZE": 64,
    "EPOCHS": 50,
    "LR": 1e-3,
    "SEED": 42
}

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(CONFIG['SEED'])

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# UPDATE THIS PATH TO YOUR DATASET LOCATION
DATASET_ROOT = '/content/drive/MyDrive/HandSpeak.ai/dataset_v2_mobile/landmarks'

## 2. Preprocessing & Augmentation Engine
Logic to normalize hands and simulate variance.

In [ ]:
class HandAugmentor:
    """Applies random rotation, scaling, and noise to landmarks."""
    
    @staticmethod
    def rotate_frame(frame, angle_deg):
        rad = np.radians(angle_deg)
        c, s = np.cos(rad), np.sin(rad)
        # Rotate around Z-axis (2D plane of screen)
        R = np.array([[c, -s, 0], [s, c, 0], [0, 0, 1]])
        return np.dot(frame, R.T)

    @staticmethod
    def augment(sequence: np.ndarray) -> np.ndarray:
        # sequence: (T, 21, 3)
        
        # 1. Random Rotation (-30 to +30 degrees)
        angle = np.random.uniform(-30, 30)
        
        # 2. Random Scale (0.8x to 1.2x)
        scale = np.random.uniform(0.8, 1.2)
        
        # 3. Random Shift/Translation (Global)
        shift = np.random.uniform(-0.1, 0.1, size=(1, 3))
        
        processed = []
        for frame in sequence:
            f = HandAugmentor.rotate_frame(frame, angle)
            f = f * scale
            f = f + shift
            processed.append(f)
            
        processed = np.array(processed)
        
        # 4. Pixel Noise (Independent per frame)
        noise = np.random.normal(0, 0.005, processed.shape) # Small jitter
        return processed + noise

class Preprocessor:
    """Canonicalizes the hand: Centers Wrist, Normalizes Size, Aligns Rotation."""
    
    def __init__(self, target_frames=30):
        self.target_frames = target_frames

    def process(self, raw_sequence: List[List[List[float]]]) -> np.ndarray:
        data = np.array(raw_sequence, dtype=np.float32)
        if data.ndim == 2: data = data[np.newaxis, ...]

        # 1. Resample
        data = self._resample(data)

        # 2. Normalize
        processed_frames = [self._normalize_frame(f) for f in data]
        return np.stack(processed_frames)

    def _resample(self, data):
        T = data.shape[0]
        if T == self.target_frames: return data
        if T < self.target_frames:
            # Pad with last frame
            padding = np.tile(data[-1], (self.target_frames - T, 1, 1))
            return np.concatenate([data, padding], axis=0)
        else:
            # Uniform sample
            idx = np.linspace(0, T-1, self.target_frames, dtype=int)
            return data[idx]

    def _normalize_frame(self, frame):
        WRIST, INDEX_MCP, MIDDLE_MCP = 0, 5, 9
        
        # Center
        frame -= frame[WRIST]

        # Scale
        palm_size = np.linalg.norm(frame[MIDDLE_MCP])
        if palm_size > 1e-6: frame /= palm_size

        # Rotate Alignment (Align Wrist->Index to Y-axis)
        v = frame[INDEX_MCP]
        angle = np.arctan2(v[0], v[1])
        theta = -angle
        c, s = np.cos(theta), np.sin(theta)
        R = np.array([[c, -s, 0], [s, c, 0], [0, 0, 1]])
        frame = np.dot(frame, R.T)
        
        return frame

## 3. Data Loading & Visualization

In [ ]:
CLASSES = list("ABCDEFGHIJKLMNOPQRSTUVWXYZ")
CLASS_TO_IDX = {c: i for i, c in enumerate(CLASSES)}
IDX_TO_CLASS = {i: c for c, i in CLASS_TO_IDX.items()}

def load_dataset(root_dir):
    samples = []
    labels = []
    
    pattern = os.path.join(root_dir, "*_landmarks.json")
    files = glob(pattern)
    print(f"Searching {pattern}... Found {len(files)} files.")
    
    for fpath in files:
        try:
            with open(fpath, 'r') as f:
                data = json.load(f)
                # Support both List[Dict] and Dict formats
                if isinstance(data, dict): data = [data]
                
                for item in data:
                    if 'landmarks' in item and len(item['landmarks']) == 21:
                        samples.append(item['landmarks'])
                        labels.append(CLASS_TO_IDX[item['letter']])
        except Exception as e:
            print(f"Skipping {fpath}: {e}")
            
    return samples, labels

# Load Data
try:
    raw_samples, raw_labels = load_dataset(DATASET_ROOT)
    print(f"Total Samples: {len(raw_samples)}")
    
    # Check Balance
    counts = Counter([IDX_TO_CLASS[l] for l in raw_labels])
    plt.figure(figsize=(12, 4))
    plt.bar(counts.keys(), counts.values())
    plt.title("Class Distribution")
    plt.show()
    
except Exception as e:
    print(f"Error loading data: {e}")
    raw_samples, raw_labels = [], []

In [ ]:
def plot_hand(landmarks, title="Hand"):
    """2D Projection Plot"""
    lms = np.array(landmarks)
    plt.figure(figsize=(4,4))
    plt.scatter(lms[:, 0], lms[:, 1], c='r')

    # Connections (MediaPipe Hands)
    CONNECTIONS = [
        (0,1), (1,2), (2,3), (3,4), # Thumb
        (0,5), (5,6), (6,7), (7,8), # Index
        (0,9), (9,10), (10,11), (11,12), # Middle
        (0,13), (13,14), (14,15), (15,16), # Ring
        (0,17), (17,18), (18,19), (19,20) # Pinky
    ]
    
    for start, end in CONNECTIONS:
        plt.plot([lms[start, 0], lms[end, 0]], [lms[start, 1], lms[end, 1]], 'b-')

    plt.gca().invert_yaxis() # Image coord system
    plt.title(title)
    plt.axis('equal')
    plt.show()

# Visualize a sample
if raw_samples:
    print("Raw Sample:")
    plot_hand(raw_samples[0], title=f"Sample {IDX_TO_CLASS[raw_labels[0]]}")
    
    # Visualize Preprocessed
    prep = Preprocessor()
    processed = prep.process([raw_samples[0]]) # Creates 30 frames
    print("Preprocessed Frame 0:")
    plot_hand(processed[0], title="Normalized")

## 4. Dataset & Model

In [ ]:
class ASLDataset(Dataset):
    def __init__(self, samples, labels, augment=False):
        self.samples = samples
        self.labels = labels
        self.augment = augment
        self.preprocessor = Preprocessor(target_frames=30)

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        raw = self.samples[idx]
        
        # Create Sequence Strategy
        # 1. Start with static frame
        base = np.array(raw, dtype=np.float32)
        
        # 2. Replicate to temporal window
        sequence = np.tile(base[np.newaxis, ...], (30, 1, 1))
        
        # 3. Augment (Key step for training robust models on static data)
        if self.augment:
            sequence = HandAugmentor.augment(sequence)
            
        # 4. Canonicalize (Preprocess)
        # Convert back to list for preprocessor compatibility if needed, or modify prep to take numpy
        # My prep.process takes List[List...]. Let's just fix prep to handle numpy.
        # Adapting here:
        processed = self.preprocessor._resample(sequence)
        processed = np.stack([self.preprocessor._normalize_frame(f) for f in processed])
        
        # Flatten: (30, 21, 3) -> (30, 63)
        x = torch.tensor(processed, dtype=torch.float32).view(30, -1)
        y = torch.tensor(self.labels[idx], dtype=torch.long)
        return x, y

class HandTransformer(nn.Module):
    def __init__(self, num_classes=26, d_model=64, nhead=4, num_layers=3, dim_feedforward=128):
        super().__init__()
        self.pos_encoder = nn.Parameter(torch.randn(1, 30, d_model) * 0.1)
        self.input_proj = nn.Linear(63, d_model)
        
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model, 
            nhead=nhead, 
            dim_feedforward=dim_feedforward,
            dropout=0.2,
            batch_first=True # (Batch, Seq, Feat)
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        
        self.head = nn.Sequential(
            nn.LayerNorm(d_model),
            nn.Linear(d_model, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, num_classes)
        )

    def forward(self, x):
        # x: (B, 30, 63)
        x = self.input_proj(x) + self.pos_encoder
        x = self.transformer(x)
        x = x.mean(dim=1) # Global Average Pooling
        return self.head(x)

## 5. Training Loop with Class Balancing

In [ ]:
# Split Data
X_train, X_val, y_train, y_val = train_test_split(
    raw_samples, raw_labels, 
    test_size=0.2, 
    stratify=raw_labels, 
    random_state=42
)

train_ds = ASLDataset(X_train, y_train, augment=True)
val_ds = ASLDataset(X_val, y_val, augment=False)

train_loader = DataLoader(train_ds, batch_size=CONFIG['BATCH_SIZE'], shuffle=True, num_workers=2)
val_loader = DataLoader(val_ds, batch_size=CONFIG['BATCH_SIZE'], shuffle=False, num_workers=2)

# Compute Class Weights
label_counts = Counter(y_train)
total_samples = sum(label_counts.values())
weights = []
for i in range(CONFIG['NUM_CLASSES']):
    if label_counts[i] > 0:
        weights.append(total_samples / (CONFIG['NUM_CLASSES'] * label_counts[i]))
    else:
        weights.append(1.0)
        
class_weights = torch.FloatTensor(weights).cuda()
print(f"Class Weights: {class_weights[:5]}...")

# Init Model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = HandTransformer(num_classes=CONFIG['NUM_CLASSES']).to(device)

optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=CONFIG['EPOCHS'])
criterion = nn.CrossEntropyLoss(weight=class_weights, label_smoothing=0.1)

# Training Loop
best_acc = 0
history = {'loss': [], 'acc': [], 'val_loss': [], 'val_acc': []}

print("Starting Training...")
for epoch in range(CONFIG['EPOCHS']):
    model.train()
    train_loss, correct, total = 0, 0, 0
    
    for X, y in train_loader:
        X, y = X.to(device), y.to(device)
        
        optimizer.zero_grad()
        out = model(X)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        _, pred = out.max(1)
        correct += pred.eq(y).sum().item()
        total += y.size(0)
        
    scheduler.step()
    
    # Validation
    model.eval()
    v_loss, v_correct, v_total = 0, 0, 0
    with torch.no_grad():
        for X, y in val_loader:
            X, y = X.to(device), y.to(device)
            out = model(X)
            loss = criterion(out, y)
            v_loss += loss.item()
            _, pred = out.max(1)
            v_correct += pred.eq(y).sum().item()
            v_total += y.size(0)
            
    # Stats
    t_acc = 100 * correct / total
    v_acc = 100 * v_correct / v_total
    history['loss'].append(train_loss / len(train_loader))
    history['acc'].append(t_acc)
    history['val_loss'].append(v_loss / len(val_loader))
    history['val_acc'].append(v_acc)
    
    print(f"Epoch {epoch+1:02d} | Loss: {train_loss/len(train_loader):.4f} | Train Acc: {t_acc:.2f}% | Val Acc: {v_acc:.2f}%")
    
    if v_acc > best_acc:
        best_acc = v_acc
        torch.save(model.state_dict(), "best_model.pth")

print(f"Done! Best Validation Acc: {best_acc:.2f}%")

## 6. Evaluation & Analysis

In [ ]:
# Plot History
plt.figure(figsize=(10, 4))
plt.subplot(1,2,1)
plt.plot(history['loss'], label='Train')
plt.plot(history['val_loss'], label='Val')
plt.title('Loss')
plt.legend()
plt.subplot(1,2,2)
plt.plot(history['acc'], label='Train')
plt.plot(history['val_acc'], label='Val')
plt.title('Accuracy')
plt.legend()
plt.show()

# Confusion Matrix
model.load_state_dict(torch.load("best_model.pth"))
model.eval()
all_preds = []
all_targets = []

with torch.no_grad():
    for X, y in val_loader:
        X, y = X.to(device), y.to(device)
        out = model(X)
        _, pred = out.max(1)
        all_preds.extend(pred.cpu().numpy())
        all_targets.extend(y.cpu().numpy())

plt.figure(figsize=(12, 10))
cm = confusion_matrix(all_targets, all_preds)
sns.heatmap(cm, annot=True, fmt='d', xticklabels=CLASSES, yticklabels=CLASSES, cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

print(classification_report(all_targets, all_preds, target_names=CLASSES))

## 7. Export ONNX

In [ ]:
dummy = torch.randn(1, 30, 63).to(device)
torch.onnx.export(
    model,
    dummy,
    "asl_model_v2.onnx",
    input_names=['input'],
    output_names=['output'],
    dynamic_axes={'input': {0: 'batch'}, 'output': {0: 'batch'}}
)
print("Model exported as asl_model_v2.onnx")